In [1]:
import pandas as pd
import io
import numpy as np
import csv

In [84]:
train = pd.read_csv('training_data.csv')
judging = pd.read_csv('contestant_judgment.csv')

In [33]:
train.head()

,ID,User,Text,Sentiment
0,864192,Carly_FTS,I *heart* filling up @dennisschaub desk 1 it...,1
1,523691,Open_Sourcing,"#SocioMat - people create prettier, younger an...",1
2,584154,xxcharlx,no way i dont want the tour to end,0
3,1527961,andreapuddu,@HemalRadia Hi Amazing Brother! Sending Limitl...,1
4,28609,umbec,@flockmaster they are chocolate,1


# DATA PROCESSING

In [4]:
import nltk
import re
nltk.download('punkt')
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem.wordnet import WordNetLemmatizer
nltk.download('wordnet')
from sklearn.feature_extraction.text import CountVectorizer

[nltk_data] Downloading package punkt to /Users/Jenny/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/Jenny/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/Jenny/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [18]:
print(len(train.at[100000, 'Text']))
print(train.at[100000, 'Text'])

86
@SorenBlackgaard And I think you meant to say, &quot;...to eclipse the Eclipse?&quot; 


In [85]:
stop_words = set(stopwords.words('english'))
filtered_sent = []
#filtered_phrase = []

for index, row in train.iterrows():
    tokenized_words = []
    # Remove @s
    train.at[index, 'Text'] = re.sub(r"@[A-z]+", " ", train.at[index, 'Text'])
    #tokenized_words = re.sub(r"@[A-z]+", " ", train.at[index, 'Text'])
    
    # Remove punct
    train.at[index, 'Text'] = re.sub(r"\W", " ", train.at[index, 'Text'])
    #tokenized_words = re.sub(r"\W", " ", tokenized_words)

    # Remove numbers
    train.at[index, 'Text'] = re.sub(r"\d+", "", train.at[index, 'Text'])
   # tokenized_words = re.sub(r"\d+", "", tokenized_words)

    # Remove spaces
    train.at[index, 'Text'] = re.sub(r"\s+", " ", train.at[index, 'Text'])
    #tokenized_words = re.sub(r"\s+", " ", tokenized_words)
   
    # Make lowercase
    train.at[index, 'Text'] = str(train.at[index, 'Text']).lower()
    #tokenized_words = str(tokenized_words).lower()
    
    # Tokenize
    #train.at[index, 'Text'] = nltk.word_tokenize(train.at[index, 'Text'])
    tokenized_words = nltk.word_tokenize(train.at[index, 'Text'])
    
    first_word=True
    
    filtered_sent.append([])
#    filtered_phrase.append([])
    
    for w in tokenized_words:
        if w not in stop_words:
            filtered_sent[index].append(w)
#        if not first_word:
#            filtered_phrase[index].append(prev + " " + w)
#        else:
#            first_word = False
#        prev = w
   
   # train[index].append(filtered_sent)
    #train[index].append(filtered_phrase)
    #filtered_sent = []
    #filtered_phrase = []


train.head()

,ID,User,Text,Sentiment
0,864192,Carly_FTS,i heart filling up desk it means sales amp it ...,1
1,523691,Open_Sourcing,sociomat people create prettier younger and b...,1
2,584154,xxcharlx,no way i dont want the tour to end,0
3,1527961,andreapuddu,hi amazing brother sending limitless love you...,1
4,28609,umbec,they are chocolate,1


# word

In [86]:
print(filtered_sent[1])
print(filtered_phrase[0])

['sociomat', 'people', 'create', 'prettier', 'younger', 'better', 'looking', 'avatars']
['i heart', 'heart filling', 'filling up', 'up desk', 'desk it', 'it means', 'means sales', 'sales amp', 'amp it', 'it s', 's off', 'off my', 'my desk']


# phrase

In [87]:
lemmatizer = WordNetLemmatizer()
i = 0
for words in filtered_sent:
    newWords = []
    
    for word in words:
        newWords.append(lemmatizer.lemmatize(word, pos = 'v'))
     
    filtered_sent[i] = " ".join(newWords)
    i += 1
    #train.at[index, 'Text'] = " ".join(newWords)

In [89]:
print(filtered_sent[0])
train.head()

heart fill desk mean sales amp desk


,ID,User,Text,Sentiment
0,864192,Carly_FTS,i heart filling up desk it means sales amp it ...,1
1,523691,Open_Sourcing,sociomat people create prettier younger and b...,1
2,584154,xxcharlx,no way i dont want the tour to end,0
3,1527961,andreapuddu,hi amazing brother sending limitless love you...,1
4,28609,umbec,they are chocolate,1


In [90]:
new_column = pd.DataFrame({'BagofWords': filtered_sent})
train = train.merge(new_column, right_index = True, left_index=True)
train.head()

,ID,User,Text,Sentiment,BagofWords
0,864192,Carly_FTS,i heart filling up desk it means sales amp it ...,1,heart fill desk mean sales amp desk
1,523691,Open_Sourcing,sociomat people create prettier younger and b...,1,sociomat people create prettier younger better...
2,584154,xxcharlx,no way i dont want the tour to end,0,way dont want tour end
3,1527961,andreapuddu,hi amazing brother sending limitless love you...,1,hi amaze brother send limitless love way twitt...
4,28609,umbec,they are chocolate,1,chocolate


In [91]:
#print(train.at[1,'Text'])

bagofwords = train.BagofWords.tolist()
sentiment = train.Sentiment.tolist()
print(bagofwords[1])

sociomat people create prettier younger better look avatars


In [92]:
cv = CountVectorizer(max_features=750)
cv2 = CountVectorizer(max_features=750, ngram_range=(2, 3))

x_1 = cv.fit_transform(train['BagofWords']).toarray()
x_2 = cv2.fit_transform(train['Text']).toarray()
y = train['Sentiment'].values

header_1 = cv.get_feature_names()
header_2 = cv2.get_feature_names()
output_1 = pd.DataFrame(x_1, columns = header)
output_2 = pd.DataFrame(x_2, columns = header_2)
output_1 = output_1.merge(output_2, right_index=True, left_index=True)
output_1['Sentiment'] = train['Sentiment']

output_1.head()


ValueError: Shape of passed values is (1000000, 1000), indices imply (1000000, 65)

In [81]:
train.tail()

,ID,User,Text,Sentiment,BagofWords
5,1284501,rt_nuggets,just got home and from what i ve heard my lak...,1,get home hear lakers beat nuggets happy
6,971375,Sweetcakes16,yup yup especially when its i t,0,yup yup especially
7,1597758,alanakf,i have a new love matt lanter of absolutely ad...,1,new love matt lanter absolutely adore ah swoon...
8,14016,MLM216,graham s car looked really quot snakey quot on...,0,graham car look really quot snakey quot replay
9,1408251,Tickle_EAC,why some people make me be evil towards them i...,0,people make evil towards hate
